In [5]:
import os
import numpy as np
import json
import copy
import util_cochlea
# import concurrent.futures

2025-04-03 22:51:09.724357: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-03 22:51:10.251332: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-03 22:51:15.497810: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# Looad cochlea config parameters

# dir_model = 'models/tensorflow2/arch01'
# fn_config = os.path.join(dir_model, 'config.json')
# with open(fn_config, 'r') as f:
#     CONFIG = json.load(f)

# # kwargs_cochlea = CONFIG['kwargs_cochlea']

#### kwargs_cochlea from models/tensorflow2/arch01

# kwargs_cochlea =  {'config_filterbank': {'max_hi': 20000.0,
# 'min_lo': 30.0,
# 'mode': 'half_cosine_filterbank',
# 'num_cf': 39},
# 'config_subband_processing': {'power_compression': 0.3, 'rectify': True},
# 'kwargs_fir_lowpass_filter_input': {},
# 'kwargs_fir_lowpass_filter_output': {'cutoff': 4000,
# 'numtaps': 4097,
# 'window': ['kaiser', 5.0]},
# 'sr_cochlea': 48000,
# 'sr_input': 48000,
# 'sr_output': 8000}

# dur = 2 # 2 seconds
# sr = 48000
# y = audio_data[:48000].reshape(1, sr, dur)
# # Cochlear model for ear index 0
# y0, _ = util_cochlea.cochlea(y[..., 0], **copy.deepcopy(kwargs_cochlea))
# # Cochlear model for ear index 1
# y1, _ = util_cochlea.cochlea(y[..., 1], **copy.deepcopy(kwargs_cochlea))

# y_out = np.concatenate([y0.numpy(), y1.numpy()], axis=0)
# y_out.shape

In [ ]:
# create a function to process the audio based on above steps, taking input as audio_data and output as y_out

def cochlea_processing(audio_path):

    dur = 2 # 2 seconds
    sr = 48000

    audio_data = np.load(os.path.join(audio_path), allow_pickle=True)
    print(f"File: {audio_path}, shape: {audio_data.shape} \n\n")
    y = audio_data[:48000].reshape(1, sr, dur)


    kwargs_cochlea =  {'config_filterbank': {'max_hi': 20000.0,
   'min_lo': 30.0,
   'mode': 'half_cosine_filterbank',
   'num_cf': 39},
  'config_subband_processing': {'power_compression': 0.3, 'rectify': True},
  'kwargs_fir_lowpass_filter_input': {},
  'kwargs_fir_lowpass_filter_output': {'cutoff': 4000,
   'numtaps': 4097,
   'window': ['kaiser', 5.0]},
  'sr_cochlea': 48000,
  'sr_input': 48000,
  'sr_output': 8000}
    

    # Cochlear model for ear index 0
    y0, _ = util_cochlea.cochlea(y[..., 0], **copy.deepcopy(kwargs_cochlea))
    # Cochlear model for ear index 1
    y1, _ = util_cochlea.cochlea(y[..., 1], **copy.deepcopy(kwargs_cochlea))

    y_out = np.concatenate([y0.numpy(), y1.numpy()], axis=0)
    return y_out

In [7]:
audio_path = '/mnt/lustre/work/macke/mwe234/datasets/simulated/500_spatial_360_7'
output_dir = '/mnt/lustre/work/macke/mwe234/datasets/simulated/cochlea_500_spatial_360_7'
os.makedirs(output_dir, exist_ok=True)

audio_files = [f for f in os.listdir(audio_path) if f.endswith('.npy')]

def process_audio(file):
    audio = os.path.join(audio_path, file)
    y_out = cochlea_processing(audio)
    
    np.save(os.path.join(output_dir, file), y_out)

# with concurrent.futures.ThreadPoolExecutor() as executor:
#     executor.map(process_audio, audio_files)


In [10]:
# load npy /mnt/lustre/work/macke/mwe234/datasets/simulated/500_spatial_360_7/sound_178099-9-0-0_azim160_elev20.npy
import numpy as np
y_out = np.load('/mnt/lustre/work/macke/mwe234/datasets/simulated/cochlea_500_spatial_360_7/sound_178099-9-0-0_azim160_elev20.npy')
y_out.shape

(2, 39, 8000)

In [12]:
# file = 'sound_178099-9-0-0_azim160_elev20.npy'
file = audio_files[0]
process_audio(file)

[cochlea] converting audio to subbands using half_cosine_filterbank
[cochlea] half-wave rectified subbands
[tf_fir_resample] interpreted `tensor_input.shape` as [batch, freq=39, time=48000]
[tf_fir_resample] `kwargs_fir_lowpass_filter`: {'cutoff': 4000, 'numtaps': 4097, 'window': ['kaiser', 5.0]}
[fir_lowpass_filter] sr_filt = 48000 Hz
[fir_lowpass_filter] numtaps = 4097 samples
[fir_lowpass_filter] fir_dur = 0.08533333333333333 seconds
[fir_lowpass_filter] cutoff = 4000 Hz
[fir_lowpass_filter] window = ('kaiser', 5.0)
[cochlea] resampled subbands from 48000 Hz to 8000 Hz with filter: {'cutoff': 4000, 'numtaps': 4097, 'window': ['kaiser', 5.0]}
[cochlea] half-wave rectified resampled subbands
[cochlea] applied 0.3 power compression to subbands
[cochlea] converting audio to subbands using half_cosine_filterbank
[cochlea] half-wave rectified subbands
[tf_fir_resample] interpreted `tensor_input.shape` as [batch, freq=39, time=48000]
[tf_fir_resample] `kwargs_fir_lowpass_filter`: {'cutoff'